# Data Model for predicting the sales of a given item for a given week - Using linear regression

## Data
1. train_data.csv - A dataset with the daily sales of each item
2. validation_data.csv - A dataset of the weekly sales of a selected number of items
3. test_data.csv - A dataset to be completed by perdicted values of sales

### Step 1: Organizing the data
Create a dataset with the weekly sales (named *weekly_data.csv*) of the items from the train_data.csv file

In [ ]:
# Required dependencies - Data filtering
from csv import DictReader, DictWriter
from datetime import datetime as dt
from datetime import timedelta as td

In [ ]:
# Required libraries for model training
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

### Data Filtering and Preparation
This following cell includes data filtering functions. It filers out the data from the train_data.csv file and
writes the filtered data to the weekly_data.csv file in the format that is required by the 
model training libraries

In [ ]:

# A function to filter out the items in each week

def filterItems(train_dataset_file:str, start_date:str, end_date:str) -> dict:
    with open(train_dataset_file) as f:
        data = DictReader(f)

        start_date = dt.strptime(start_date, "%m/%d/%Y")
        end_date = dt.strptime(end_date, "%m/%d/%Y")
        week = td(days=7)
        days_td = (end_date-start_date)
        num_weeks = int((((days_td.total_seconds()//3600)//24)//7))

        weeks = {f'w{j+1}':{'start':start_date+j*week,'items':{}} for j in range(num_weeks)}

        for row in data:
            date = dt.strptime(row['DateID'], '%m/%d/%Y')
            for key, value in weeks.items():

                start,items = value['start'], value['items']
    
                if start<=date<=start+week*4:
                    # If the item was added earlier
                    if row['ItemCode'] in items.keys():
                        items[row['ItemCode']]['WeeklySales']+=int(row['DailySales'])
                    # If the item was not added
                    else:
                        items[row['ItemCode']] = {
                            'CategoryCode':row['CategoryCode'],
                            'ItemCode':row['ItemCode'],
                            'WeeklySales':0
                        }
    return weeks


# A function to write data into a new csv file
def writeWeeklyData(dataset:dict, filename:str) -> None:
    contents = []
    for key, value in dataset.items():
        allitems = value['items']
        
        for _, items in allitems.items():

            content = {
                'CategoryCode': int(items['CategoryCode'].split("_")[-1]),
                'ItemCode': int(items['ItemCode']),
                'Week':int(key[1::]),
                'WeeklySales':int(items['WeeklySales'])
            }
            contents.append(content)
    
    with open(filename, 'w', newline='') as f:
        fieldnames = ['CategoryCode', 'ItemCode', 'Week', 'WeeklySales']
        writer = DictWriter(f, fieldnames=fieldnames)

        # Write the header
        writer.writeheader()
        writer.writerows(contents)

# Get the data
weeks = filterItems('./train_data.csv', '10/01/2021', '02/13/2022')
writeWeeklyData(weeks, './weekly_data.csv')


### Model creation and training
The following cells creates and trains a Linear Regression model on the given training data and predicts the values for the upcoming 4 weeks

In [ ]:
# Read data from the weekly_data file
data = pd.read_csv('./weekly_data.csv')
data

In [19]:
# A model using linear regression
model_LR = LinearRegression()
model_LR.fit(data[['CategoryCode', 'ItemCode', 'Week']].values, data.WeeklySales)

LinearRegression()

### Predicting using the created model
As the linear regression model is created using the above cells, now it can be used to predict the weekly sales for the upcoming weeks.

The following cells tests the model for the test data in the *test_data.csv* file and write the new values to a file named *predicted_test_data.csv*

In [21]:
def PredictValues(input_filename:str, output_filename:str, offset_weeks:int, Datamodel) -> None:
    # Read the test data and calculate predict the weekly sales
    with open (input_filename, newline='') as inputfile:
        data = DictReader(inputfile)
        contents = []

        for row in data:
            # Format the data as required
            categorycode = int(row['CategoryCode'].split('_')[-1])
            itemcode = int(row['ItemCode'])
            week = int(row['Week'][1::])+offset_weeks
            
            # Run the model to predict the weeklysales
            predictedsales = Datamodel.predict([[categorycode, itemcode, week]])

            # Create dict object to be written to the csv file 
            content = {
                'CategoryCode': row['CategoryCode'],
                'ItemCode': row['ItemCode'],
                'Week': row['Week'],
                'PredictedSales': round(predictedsales[0])
            }
        
            contents.append(content)
    
    # Write the predicted sales to a new file
    with open(output_filename, 'w', newline='') as outputfile:
        fieldnames = ['CategoryCode', 'ItemCode', 'Week', 'PredictedSales']
        writer = DictWriter(outputfile, fieldnames=fieldnames)

        # write the header
        writer.writeheader()
        writer.writerows(contents)
    
    return contents

    
# Predict the values
contents = PredictValues('./test_data.csv', './predicted_test_data.csv', offset_weeks=19, Datamodel=model_LR)

### Validating and testing the model
The following code 